In [1]:
import os
import sys
import pandas as pd
from dotenv import load_dotenv
import snowflake.connector

# Load environment variables from a .env file
load_dotenv()

def connect_to_snowflake():
    """Establishes a connection to Snowflake using environment variables."""
    try:
        conn = snowflake.connector.connect(
            user=os.getenv('SNOWFLAKE_USER'),
            password=os.getenv('SNOWFLAKE_PASSWORD'),
            account=os.getenv('SNOWFLAKE_ACCOUNT'),
            warehouse=os.getenv('SNOWFLAKE_WAREHOUSE', 'COMPUTE_WH'),
            database='INCREMENTALITY',
            schema='INCREMENTALITY_RESEARCH'
        )
        print("✅ Connected to Snowflake")
        return conn
    except snowflake.connector.Error as e:
        print(f"❌ Could not connect to Snowflake: {e}", file=sys.stderr)
        return None

def build_user_panel_query(start_date=None, end_date=None):
    """
    Builds the SQL query for the user-week panel.
    If start_date and end_date are provided, it adds a WHERE clause to filter by date.
    Otherwise, it queries the entire history.
    """
    
    # Conditionally create the WHERE clauses
    if start_date and end_date:
        clicks_where_clause = f"WHERE OCCURRED_AT >= '{start_date}' AND OCCURRED_AT < '{end_date}'"
        purchases_where_clause = f"WHERE PURCHASED_AT >= '{start_date}' AND PURCHASED_AT < '{end_date}'"
    else:
        clicks_where_clause = ""
        purchases_where_clause = ""

    query = f"""
    WITH
    -- Step 1: Aggregate clicks per user for the specified period.
    CLICKS_WEEKLY AS (
        SELECT
            USER_ID,
            DATE_TRUNC('WEEK', OCCURRED_AT) AS week,
            COUNT(DISTINCT INTERACTION_ID) AS click_count
        FROM CLICKS
        {clicks_where_clause}
        GROUP BY USER_ID, week
    ),

    -- Step 2: Aggregate purchases and revenue per user for the specified period.
    PURCHASES_WEEKLY AS (
        SELECT
            USER_ID,
            DATE_TRUNC('WEEK', PURCHASED_AT) AS week,
            COUNT(DISTINCT PURCHASE_ID) AS purchase_count,
            COALESCE(SUM(QUANTITY * UNIT_PRICE), 0) AS total_revenue_cents
        FROM PURCHASES
        {purchases_where_clause}
        GROUP BY USER_ID, week
    )

    -- Final Step: Join user clicks and purchases for the period.
    SELECT
        COALESCE(c.week, p.week) AS week,
        COALESCE(c.user_id, p.user_id) AS user_id,
        COALESCE(c.click_count, 0) AS clicks,
        COALESCE(p.purchase_count, 0) AS purchases,
        (COALESCE(p.total_revenue_cents, 0) / 100)::DECIMAL(18, 2) AS revenue_dollars
        
    FROM CLICKS_WEEKLY AS c
    FULL OUTER JOIN PURCHASES_WEEKLY AS p
        ON c.user_id = p.user_id AND c.week = p.week
    ORDER BY
        user_id,
        week;
    """
    return query

def fetch_user_panel_data(conn, query):
    """Executes a query and returns the result as a Pandas DataFrame."""
    print("Executing query...")
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        if cursor.description:
            results = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]
            df = pd.DataFrame(results, columns=columns)
            print(f"✅ Query successful. Fetched {len(df):,} rows.")
            return df
        return pd.DataFrame()
    except snowflake.connector.Error as e:
        print(f"\n❌ ERROR executing query: {e}", file=sys.stderr)
        return pd.DataFrame()
    finally:
        cursor.close()

def process_and_save_data(df, filename):
    """Processes and saves the DataFrame to a Parquet file."""
    if df.empty:
        print("No data to save.")
        return

    try:
        df.columns = [col.lower() for col in df.columns]
        df.to_parquet(filename, index=False, engine='pyarrow')
        print(f"✅ Data successfully processed and saved to '{filename}'")
    except Exception as e:
        print(f"❌ Failed to save data to '{filename}': {e}", file=sys.stderr)

def main():
    """
    Main function to orchestrate the data extraction and saving process.
    """
    # --- CONFIGURATION ---
    # Choose ONE of the following configurations.

    # OPTION 1: Full History (all weeks)
    START_DATE = None
    END_DATE = None
    OUTPUT_FILENAME = "user_panel_full_history.parquet"
    
    # # OPTION 2: Specific Date Range (e.g., first two weeks of July 2025)
    # START_DATE = '2025-07-01'
    # END_DATE = '2025-07-15'   # End date is exclusive
    # OUTPUT_FILENAME = "user_panel_2_weeks_july_2025.parquet"
    
    # ---------------------

    if START_DATE:
        print(f"--- Generating User-Week Panel for {START_DATE} to {END_DATE} ---")
    else:
        print("--- Generating User-Week Panel for ALL WEEKS ---")

    conn = connect_to_snowflake()
    if not conn:
        sys.exit(1)

    try:
        # 1. Build the appropriate SQL query
        query = build_user_panel_query(START_DATE, END_DATE)
        
        # 2. Fetch data from Snowflake
        user_panel_df = fetch_user_panel_data(conn, query)
        
        # 3. Process and save the data
        process_and_save_data(user_panel_df, OUTPUT_FILENAME)

    finally:
        if conn and not conn.is_closed():
            conn.close()
            print("✅ Snowflake connection closed.")

if __name__ == "__main__":
    main()

--- Generating User-Week Panel for ALL WEEKS ---
✅ Connected to Snowflake
Executing query...
✅ Query successful. Fetched 32,060,768 rows.
✅ Data successfully processed and saved to 'user_panel_full_history.parquet'
✅ Snowflake connection closed.


In [20]:
import polars as pl
from tabulate import tabulate

# --- Helper Function (Modified to work with Polars) ---
def show_table(df, title=""):
    """Prints a Polars DataFrame in a formatted grid table by converting it to Pandas."""
    if title:
        print(f"\n{title}")
        print("="*len(title))
    # Tabulate works best with Pandas, so we convert just for printing
    print(tabulate(df.to_pandas(), headers='keys', tablefmt='grid', showindex=False))

# --- Prerequisite: Load and Prepare Data ---
try:
    parquet_filename = 'user_panel_full_history.parquet'
    
    print(f"--- Loading and preparing data from '{parquet_filename}' ---")
    
    # Load data using Polars and chain data preparation steps
    df_analysis = pl.read_parquet(parquet_filename).with_columns(
        
        # --- FIX: Cast the Decimal column to Float64 at the source ---
        # This prevents overflow errors in all downstream aggregations (like .sum()).
        pl.col("revenue_dollars").cast(pl.Float64),
        
        # Create log-transformed variables (these work fine on the new Float64 column)
        pl.col("revenue_dollars").log1p().alias("log_revenue_plus_1"),
        pl.col("clicks").log1p().alias("log_clicks_plus_1")
    )
    print(f"✅ Data loaded and prepared successfully.")

except FileNotFoundError:
    print(f"❌ ERROR: The file '{parquet_filename}' was not found.")
    print("   Please run the data generation script first or check the filename.")
    df_analysis = None

# --- Main EDA Block ---
# All analysis is contained within this block to ensure it only runs if data loading was successful.
if df_analysis is not None and len(df_analysis) > 0:

    # --- EDA 1: High-Level Overview ---
    print("\n--- EDA 1: High-Level Overview ---")
    num_rows, num_cols = df_analysis.shape
    
    # Use Polars expressions to get unique counts
    summary_stats = df_analysis.select(
        pl.len().alias("Total Observations (User-Weeks)"),
        pl.count().alias("Total Columns"), # In Polars, pl.count() counts columns
        pl.col("user_id").n_unique().alias("Unique Users"),
        pl.col("week").n_unique().alias("Unique Weeks")
    ).melt().rename({"variable": "Metric", "value": "Value"})
    
    show_table(summary_stats, "Dataset Dimensions")
    
    # Get schema information
    schema_df = pl.DataFrame({
        "column": df_analysis.columns,
        "data_type": df_analysis.dtypes
    })
    show_table(schema_df, "Column Data Types")

    # --- EDA 2: Distribution of Core Metrics ---
    print("\n--- EDA 2: Distribution of User-Week Metrics ---")
    
    # Get descriptive statistics using Polars
    desc_stats = df_analysis.select(['clicks', 'purchases', 'revenue_dollars']).describe()
    show_table(desc_stats, "Overall Distribution of Key Metrics (per User per Week)")
               
    # Calculate zero revenue percentage using Polars expression
    zero_revenue_pct = df_analysis.select(
        (pl.col('revenue_dollars') == 0).mean() * 100
    ).item()
    print(f"\nUser-Weeks with Zero Revenue: {zero_revenue_pct:.2f}% of observations have zero revenue.")
    
    # Filter for purchasing users and get their stats
    df_purchasers = df_analysis.filter(pl.col('revenue_dollars') > 0)
    show_table(df_purchasers.select(['clicks', 'purchases', 'revenue_dollars']).describe(),
               "Distribution for Purchasing User-Weeks ONLY")

    # --- EDA 3: Correlation Analysis ---
    print("\n--- EDA 3: Correlation Between Clicks and Revenue ---")
    
    # Polars calculates correlations pairwise. For a matrix, it's often easiest to bridge to Pandas.
    corr_raw_pd = df_analysis.select(['clicks', 'purchases', 'revenue_dollars']).to_pandas().corr()
    show_table(pl.from_pandas(corr_raw_pd.reset_index()), "Correlation Matrix (Raw Values)")
    
    corr_log_pd = df_analysis.select(['log_clicks_plus_1', 'log_revenue_plus_1']).to_pandas().corr()
    show_table(pl.from_pandas(corr_log_pd.reset_index()), "Correlation Matrix (Log-Transformed Values)")
    
    print("\nInterpretation: Correlation shows the strength of the linear relationship between user clicks")
    print("and their spending. The log-transformed value is often more stable for skewed data like revenue.")

    # --- EDA 4: Top User Summary ---
    print("\n--- EDA 4: Top User Analysis (Aggregated Across the Period) ---")
    
    # Perform aggregation using Polars' group_by().agg() syntax
    user_summary = (
        df_analysis.group_by('user_id')
        .agg(
            pl.col('clicks').sum().alias('total_clicks'),
            pl.col('purchases').sum().alias('total_purchases'),
            pl.col('revenue_dollars').sum().alias('total_revenue'),
            pl.col('week').n_unique().alias('weeks_active')
        )
        .sort('total_revenue', descending=True)
        .head(15)
    )
    
    # Format the revenue column for better readability using `map_elements`
    user_summary = user_summary.with_columns(
        pl.col('total_revenue').map_elements(lambda x: f"${x:,.2f}", return_dtype=pl.String)
    )
    
    show_table(user_summary, "Top 15 Users by Total Revenue")
    print("\nInterpretation: This table helps identify if a small number of 'whale' users are responsible")
    print("for a large portion of the period's total revenue.")

else:
    print("\nCould not perform EDA because the DataFrame is empty or could not be loaded.")

# --- EDA 5: Identifying Potential Holdout & User Groups ---
print("\n--- EDA 5: Identifying Potential Holdout & User Groups ---")

# To analyze lifetime behavior, we first need to aggregate the panel to the user level.
# This creates a summary for every unique user in the dataset.
print("   Aggregating data to user level for analysis...")
user_summary_full = (
    df_analysis.group_by('user_id')
    .agg(
        pl.col('clicks').sum().alias('total_clicks'),
        pl.col('purchases').sum().alias('total_purchases'),
        pl.col('revenue_dollars').sum().alias('total_revenue'),
        pl.col('week').min().alias('first_seen_week')
    )
)
total_users = len(user_summary_full)
print(f"   Analyzed {total_users:,} unique users.")

# --- Analysis 1: Find "Organic Purchasers" (Potential Holdout Group) ---
# These are users who convert without any recorded clicks. They are crucial for
# understanding the baseline conversion rate of your platform.

organic_purchasers = user_summary_full.filter(
    (pl.col("total_clicks") == 0) & (pl.col("total_purchases") > 0)
)
count_organic = len(organic_purchasers)

# --- Analysis 2: Find "Late Joiners" ---
# These are users who were not present at the start of the data period.
# A large number of late joiners means the panel is not "balanced", which is
# important to know for certain types of models.

first_week_in_dataset = df_analysis.select(pl.col("week").min()).item()

late_joiners = user_summary_full.filter(
    pl.col("first_seen_week") > first_week_in_dataset
)
count_late = len(late_joiners)

# --- Present the Results ---

summary_data = pl.DataFrame({
    "User Segment": [
        "Total Unique Users",
        "Organic Purchasers (Zero Clicks)",
        "Late Joiners (Joined after first week)"
    ],
    "User Count": [
        total_users,
        count_organic,
        count_late
    ]
}).with_columns(
    # Calculate percentage of total for each group
    (pl.col("User Count") / total_users * 100).round(2).alias("Percentage of Total")
)

show_table(summary_data, "Summary of Key User Segments")

print("\nInterpretation:")
print(" - Organic Purchasers: This group represents your baseline organic conversion. They buy without")
print("   clicking on tracked ads. They are a perfect 'never-treated' control group for analysis.")
print(" - Late Joiners: This shows how many users are new to the platform within the dataset's timeframe.")
print("   A high number indicates a growing user base but can complicate longitudinal analysis.")

# --- Deeper Dive into the Organic Purchasers ---
if count_organic > 0:
    # Calculate the economic value of this 'holdout' group
    organic_purchaser_stats = organic_purchasers.select(
        pl.col("total_purchases").mean().alias("avg_total_purchases"),
        pl.col("total_revenue").mean().alias("avg_total_revenue"),
        pl.col("total_revenue").sum().alias("sum_total_revenue")
    )
    
    # Format the sum of revenue for readability
    formatted_stats = organic_purchaser_stats.with_columns(
        pl.col('sum_total_revenue').map_elements(lambda x: f"${x:,.0f}", return_dtype=pl.String)
    )
    
    show_table(formatted_stats, "Behavior of 'Organic Purchaser' Segment")
    print("\nInterpretation:")
    print(" - This table shows the average behavior and total economic value of users who purchase")
    print("   without ever clicking on a tracked advertisement.")

--- Loading and preparing data from 'user_panel_full_history.parquet' ---
✅ Data loaded and prepared successfully.

--- EDA 1: High-Level Overview ---


/var/folders/b7/1tvk5qmx0ds9c6gk2lrlhv380000gn/T/ipykernel_60208/1927247169.py:48: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("Total Columns"), # In Polars, pl.count() counts columns
/var/folders/b7/1tvk5qmx0ds9c6gk2lrlhv380000gn/T/ipykernel_60208/1927247169.py:51: DeprecationWarning: `DataFrame.melt` is deprecated; use `DataFrame.unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  ).melt().rename({"variable": "Metric", "value": "Value"})



Dataset Dimensions
+---------------------------------+----------+
| Metric                          |    Value |
+=================================+==========+
| Total Observations (User-Weeks) | 31657200 |
+---------------------------------+----------+
| Total Columns                   | 31657200 |
+---------------------------------+----------+
| Unique Users                    |  9183985 |
+---------------------------------+----------+
| Unique Weeks                    |       27 |
+---------------------------------+----------+

Column Data Types
+--------------------+------------------------------------------+
| column             | data_type                                |
+====================+==========================================+
| week               | Datetime(time_unit='ns', time_zone=None) |
+--------------------+------------------------------------------+
| user_id            | String                                   |
+--------------------+--------------------------

In [24]:
import os
import sys
import polars as pl

# --- 1. Set up rpy2 Environment for fixest ---
# --- 1. Set up rpy2 Environment for fixest ---
# This function isolates the rpy2 setup.
def setup_rpy2():
    """Initializes the rpy2 environment and returns key objects."""
    try:
        import rpy2.robjects as ro
        from rpy2.robjects.packages import importr
        from rpy2.robjects import pandas2ri
        from rpy2.robjects.conversion import localconverter

        # --- LINE REMOVED ---
        # pandas2ri.activate() # This line was deprecated and is not needed.
        # The 'with localconverter(...)' context manager in the modeling
        # function is the correct and modern way to handle conversion.
        
        # Load R libraries
        importr('fixest')
        
        print("✅ rpy2 environment and 'fixest' library loaded successfully.")
        return True, ro, pandas2ri, localconverter
    except (ImportError, RuntimeError) as e:
        print(f"❌ ERROR: Failed to set up rpy2 or load R libraries: {e}", file=sys.stderr)
        print("   Please ensure R is installed and `fixest` is available in the R environment.", file=sys.stderr)
        return False, None, None, None

# --- 2. Load, Prepare, and Sub-sample Data ---
def load_and_prepare_data(filename, sample_fraction=1.0):
    """
    Loads user-panel data using Polars, prepares log-transformed columns,
    and optionally samples a fraction of users.
    """
    try:
        print(f"--- Loading panel data from '{filename}' ---")
        df = pl.read_parquet(filename)

        # Prepare log-transformed variables using Polars expressions
        df = df.with_columns(
            pl.col("revenue_dollars").log1p().alias("log_revenue_plus_1"),
            pl.col("clicks").log1p().alias("log_clicks_plus_1")
        )

        # Handle sub-sampling at the user level
        if sample_fraction < 1.0:
            print(f"--- Sub-sampling {sample_fraction:.0%} of users... ---")
            unique_users = df.get_column("user_id").unique()
            sampled_users = unique_users.sample(fraction=sample_fraction, shuffle=True)
            df = df.filter(pl.col("user_id").is_in(sampled_users))
            print(f"✅ Sampled data contains {df.height:,} rows from {sampled_users.len():,} users.")
        else:
            print(f"✅ Loaded and prepared full panel with {df.height:,} rows.")
        
        return df

    except FileNotFoundError:
        print(f"❌ ERROR: The file '{filename}' was not found.", file=sys.stderr)
        return None
    except Exception as e:
        print(f"❌ ERROR during data loading or preparation: {e}", file=sys.stderr)
        return None

# --- 3. Run the Fixed-Effects Model in R ---
def run_fixed_effects_model(df, ro, pandas2ri, localconverter):
    """
    Transfers the DataFrame to R and runs a two-way fixed-effects model.
    """
    print("\n--- Estimating Model: log(revenue+1) ~ log(clicks+1) | user_id + week ---")
    
    try:
        # Convert Polars to Pandas for rpy2 compatibility
        df_pd = df.to_pandas()

        # Transfer the data to the R global environment
        with localconverter(ro.default_converter + pandas2ri.converter):
            ro.globalenv['df_for_r'] = pandas2ri.py2rpy(df_pd)

        # Execute the R code for the fixest model
        ro.r("""
        library(fixest)
        df_panel <- df_for_r
        
        # Estimate the model with user and week fixed-effects.
        # Standard errors are clustered at the user level to account for
        # correlation in errors for the same user over time.
        model_user_fe <- feols(
            log_revenue_plus_1 ~ log_clicks_plus_1 | user_id + week, 
            data = df_panel, 
            vcov = ~user_id
        )
        
        # Print the results table
        print(etable(model_user_fe, digits = 4))
        """)
        
        print("\n✅ Fixed-effects model estimated successfully.")

    except Exception as e:
        print(f"❌ ERROR running the fixest model in R: {e}", file=sys.stderr)

# --- Main Execution Block ---
def main():
    """Orchestrates the loading, sampling, and modeling process."""
    
    # --- CONFIGURATION ---
    # Set the input filename and the fraction of users to sample (1.0 for all users).
    FILENAME = 'user_panel_full_history.parquet'
    SAMPLE_FRACTION = 0.25  # Use 10% of users for a quick test. Set to 1.0 for the full dataset.
    # ---------------------

    rpy2_is_ready, ro, pandas2ri, localconverter = setup_rpy2()
    if not rpy2_is_ready:
        sys.exit(1)

    df_analysis = load_and_prepare_data(FILENAME, sample_fraction=SAMPLE_FRACTION)

    if df_analysis is not None and not df_analysis.is_empty():
        run_fixed_effects_model(df_analysis, ro, pandas2ri, localconverter)
    else:
        print("Skipping model estimation due to data loading issues.")

if __name__ == "__main__":
    main()

✅ rpy2 environment and 'fixest' library loaded successfully.
--- Loading panel data from 'user_panel_full_history.parquet' ---
--- Sub-sampling 25% of users... ---


/var/folders/b7/1tvk5qmx0ds9c6gk2lrlhv380000gn/T/ipykernel_60208/249180285.py:52: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  df = df.filter(pl.col("user_id").is_in(sampled_users))


✅ Sampled data contains 7,912,264 rows from 2,295,996 users.

--- Estimating Model: log(revenue+1) ~ log(clicks+1) | user_id + week ---


/Users/pranjal/Code/topsort-incrementality/venv/lib/python3.13/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "revenue_dollars". Fall back to string conversion. The error is: <class 'decimal.Decimal'>
  warnings.warn('Error while trying to convert '


                       model_user_fe
Dependent Var.:   log_revenue_plus_1
                                    
log_clicks_plus_1 0.0282*** (0.0015)
Fixed-Effects:    ------------------
user_id                          Yes
week                             Yes
_________________ __________________
S.E.: Clustered          by: user_id
Observations               7,912,264
R2                           0.46165
Within R2                    0.00014
---
Signif. codes: 0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

✅ Fixed-effects model estimated successfully.


In [26]:
import os
import sys
import polars as pl

# --- 1. Set up rpy2 Environment ---
def setup_rpy2():
    """Initializes the rpy2 environment and returns key objects."""
    try:
        import rpy2.robjects as ro
        from rpy2.robjects.packages import importr
        from rpy2.robjects import pandas2ri
        from rpy2.robjects.conversion import localconverter
        
        # Load R libraries needed for the model
        importr('lme4')
        
        print("✅ rpy2 environment and 'lme4' library loaded successfully.")
        return True, ro, pandas2ri, localconverter
    except (ImportError, RuntimeError) as e:
        print(f"❌ ERROR: Failed to set up rpy2 or load R libraries: {e}", file=sys.stderr)
        print("   Please ensure R is installed and `lme4` is available in the R environment.", file=sys.stderr)
        return False, None, None, None

# --- 2. Load and Prepare Data ---
def load_and_prepare_data(filename, sample_fraction=1.0):
    """
    Loads user-panel data using Polars, prepares log columns, and samples users.
    """
    try:
        print(f"--- Loading panel data from '{filename}' ---")
        df = (
            pl.read_parquet(filename)
            .with_columns(
                pl.col("revenue_dollars").cast(pl.Float64),
                pl.col("revenue_dollars").log1p().alias("log_revenue_plus_1"),
                pl.col("clicks").log1p().alias("log_clicks_plus_1")
            )
        )

        if sample_fraction < 1.0:
            print(f"--- Sub-sampling {sample_fraction:.1%} of users for modeling... ---")
            unique_users = df.get_column("user_id").unique()
            sampled_users = unique_users.sample(fraction=sample_fraction, shuffle=True)
            df = df.filter(pl.col("user_id").is_in(sampled_users))
            print(f"✅ Sampled data contains {df.height:,} rows from {sampled_users.len():,} users.")
        else:
            print(f"✅ Loaded and prepared full panel with {df.height:,} rows.")
        
        return df

    except FileNotFoundError:
        print(f"❌ ERROR: The file '{filename}' was not found.", file=sys.stderr)
        return None

# --- 3. Run Mixed-Effects Model to get Beta_u ---
def run_mixed_effects_model(df, ro, pandas2ri, localconverter):
    """
    Runs a mixed-effects model in R (lme4) to estimate user-specific slopes (beta_u).
    """
    print("\n--- Preparing data for mixed-effects model ---")
    
    # 1. Filter for users with enough data to estimate a slope
    user_counts = df.group_by("user_id").len()
    users_with_enough_data = user_counts.filter(pl.col("len") > 1).get_column("user_id")
    df_clean = df.filter(pl.col("user_id").is_in(users_with_enough_data))
    
    # 2. Scale the predictor variable for model stability
    log_clicks_std = df_clean.get_column("log_clicks_plus_1").std()
    df_model_ready = df_clean.with_columns(
        ((pl.col("log_clicks_plus_1") - pl.col("log_clicks_plus_1").mean()) / log_clicks_std)
        .alias("log_clicks_scaled")
    )
    print(f"✅ Using {df_model_ready.height:,} rows from {users_with_enough_data.len():,} users with >1 observation.")
    print(f"✅ Scaled 'log_clicks' to improve model stability.")

    print("\n--- Estimating Mixed-Effects Model with `lme4` to get individual slopes (βu) ---")
    try:
        # Transfer data and scaling factor to R
        with localconverter(ro.default_converter + pandas2ri.converter):
            ro.globalenv['df_for_r'] = df_model_ready.to_pandas()
            ro.globalenv['std_log_clicks_r'] = log_clicks_std

        # Execute R code to run the model and extract coefficients
        ro.r("""
        library(lme4)
        
        # This formula estimates a fixed effect for scaled clicks, plus a random
        # intercept and slope for each user, and a random intercept for each week.
        # The '||' syntax assumes uncorrelated random effects for stability.
        lmer_model <- lmer(
            log_revenue_plus_1 ~ log_clicks_scaled + (log_clicks_scaled || user_id) + (1 | week),
            data = df_for_r,
            control = lmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e5))
        )
        
        # Extract the final (regularized) coefficients for each user
        user_coefs <- coef(lmer_model)$user_id
        
        # Get the scaled slope (beta) for each user
        beta_u_scaled <- user_coefs[, "log_clicks_scaled"]
        
        # Un-scale the beta to make it interpretable in original units
        # beta_original = beta_scaled / std_dev
        beta_u_unscaled <- beta_u_scaled / std_log_clicks_r
        
        # Create a final data frame with user_id and their unscaled beta
        results_df <- data.frame(
            user_id = rownames(user_coefs), 
            beta_u = beta_u_unscaled
        )
        """)
        
        print("\n✅ Mixed-effects model estimated successfully.")
        
        # Bring the results back into a Polars DataFrame
        with localconverter(ro.default_converter + pandas2ri.converter):
            results_pd = ro.r['results_df']
        return pl.from_pandas(results_pd)

    except Exception as e:
        print(f"❌ ERROR running the lme4 model in R: {e}", file=sys.stderr)
        return None

# --- Main Execution Block ---
def main():
    """Orchestrates the data loading and modeling process."""
    
    # --- CONFIGURATION ---
    FILENAME = 'user_panel_full_history.parquet'
    # WARNING: lme4 is very slow. A 1% sample is a good starting point.
    # Increasing this significantly will dramatically increase runtime and memory usage.
    SAMPLE_FRACTION = 0.25  # Use 1% of users.
    # ---------------------

    rpy2_is_ready, ro, pandas2ri, localconverter = setup_rpy2()
    if not rpy2_is_ready:
        sys.exit(1)

    df_analysis = load_and_prepare_data(FILENAME, sample_fraction=SAMPLE_FRACTION)

    if df_analysis is not None and not df_analysis.is_empty():
        beta_u_df = run_mixed_effects_model(df_analysis, ro, pandas2ri, localconverter)
        
        if beta_u_df is not None:
            print("\n--- Summary of Regularized User-Specific Slopes (βu) ---")
            # Use the .describe() method on the results DataFrame
            summary_table = beta_u_df.describe()
            print(summary_table)

if __name__ == "__main__":
    main()

✅ rpy2 environment and 'lme4' library loaded successfully.
--- Loading panel data from 'user_panel_full_history.parquet' ---
--- Sub-sampling 10.0% of users for modeling... ---


/var/folders/b7/1tvk5qmx0ds9c6gk2lrlhv380000gn/T/ipykernel_60208/1008257996.py:44: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  df = df.filter(pl.col("user_id").is_in(sampled_users))


✅ Sampled data contains 3,166,474 rows from 918,398 users.

--- Preparing data for mixed-effects model ---


/var/folders/b7/1tvk5qmx0ds9c6gk2lrlhv380000gn/T/ipykernel_60208/1008257996.py:65: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  df_clean = df.filter(pl.col("user_id").is_in(users_with_enough_data))


✅ Using 2,697,618 rows from 449,542 users with >1 observation.
✅ Scaled 'log_clicks' to improve model stability.

--- Estimating Mixed-Effects Model with `lme4` to get individual slopes (βu) ---


R callback write-console: In addition:   
R callback write-console: Warning message:
  
R callback write-console: In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :  
R callback write-console: 
   
R callback write-console:  Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables?
  



✅ Mixed-effects model estimated successfully.

--- Summary of Regularized User-Specific Slopes (βu) ---
shape: (9, 3)
┌────────────┬─────────────────────────────────┬───────────┐
│ statistic  ┆ user_id                         ┆ beta_u    │
│ ---        ┆ ---                             ┆ ---       │
│ str        ┆ str                             ┆ f64       │
╞════════════╪═════════════════════════════════╪═══════════╡
│ count      ┆ 449542                          ┆ 449542.0  │
│ null_count ┆ 0                               ┆ 0.0       │
│ mean       ┆ null                            ┆ -0.292076 │
│ std        ┆ null                            ┆ 0.485608  │
│ min        ┆ ext1:000006d9-686c-44a0-a494-1… ┆ -3.118796 │
│ 25%        ┆ null                            ┆ -0.618303 │
│ 50%        ┆ null                            ┆ -0.25332  │
│ 75%        ┆ null                            ┆ -0.049514 │
│ max        ┆ ext1:fffff588-b1c8-4e2c-bf84-e… ┆ 2.511748  │
└────────────┴─────────────